In [1]:
# generator_matrix_wimax.py
# Usage: python3 generator_matrix_wimax.py
# Requires: numpy

import numpy as np
import os

# ---------- CONFIG ----------
PROTO_FILE = "./wman_N0576_R34_z24.txt"   # path to the protomatrix file you provided
Z = 24   # lifting factor for this WiMAX file
OUT_DIR = "./"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------- GF(2) helpers ----------
def gf2_inverse(B):
    """Inverse of square binary matrix B (uint8). Raises ValueError if singular."""
    B = B.copy().astype(np.uint8)
    n = B.shape[0]
    inv = np.eye(n, dtype=np.uint8)
    row = 0
    for col in range(n):
        # find pivot in or below row
        pivot_rows = np.where(B[row:, col] == 1)[0]
        if pivot_rows.size == 0:
            continue
        pivot = row + pivot_rows[0]
        if pivot != row:
            B[[row, pivot]] = B[[pivot, row]]
            inv[[row, pivot]] = inv[[pivot, row]]
        # eliminate other rows
        for r in range(n):
            if r != row and B[r, col]:
                B[r] ^= B[row]
                inv[r] ^= inv[row]
        row += 1
        if row == n:
            break
    if not np.all(B == np.eye(n, dtype=np.uint8)):
        raise ValueError("Matrix not invertible.")
    return inv

# ---------- Expand protomatrix ----------
def expand_protomatrix(proto, z):
    proto = np.array(proto, dtype=int)
    M, N = proto.shape
    m = M * z
    n = N * z
    H = np.zeros((m, n), dtype=np.uint8)
    for i in range(M):
        for j in range(N):
            p = int(proto[i, j])
            if p == -1:
                continue
            r0 = i * z
            c0 = j * z
            # place shifted identity: column c0 + t has a 1 at row r0 + ((t - p) % z)
            for t in range(z):
                row_idx = r0 + ((t - p) % z)
                col_idx = c0 + t
                H[row_idx, col_idx] = 1
    return H

# ---------- Find column permutation and compute G ----------
def compute_generator_from_H(H):
    """
    Input: H (m x n) uint8
    Output: G_full (k x n) uint8 in original column ordering, H_perm (m x n) (permuted), column_order (length n)
    """
    H_orig = H.copy().astype(np.uint8)
    m, n = H_orig.shape
    k = n - m

    # Make working copy for column/row pivoting
    Hc = H_orig.copy()
    perm = np.arange(n)  # tracks column swaps: perm[new_index] = original_col_index

    row = 0
    col = 0
    pivot_cols = []
    # Greedy Gauss-Jordan with column swaps to produce m pivots
    while row < m and col < n:
        # find a column c >= col that has a 1 in row..m-1
        found = False
        for c in range(col, n):
            if np.any(Hc[row:, c] == 1):
                # swap column c into position 'col'
                if c != col:
                    Hc[:, [col, c]] = Hc[:, [c, col]]
                    perm[[col, c]] = perm[[c, col]]
                # find pivot row (first 1 at or below row)
                rows = np.where(Hc[row:, col] == 1)[0]
                pivot = row + rows[0]
                if pivot != row:
                    Hc[[row, pivot]] = Hc[[pivot, row]]
                # eliminate other rows at column 'col'
                for r in range(m):
                    if r != row and Hc[r, col]:
                        Hc[r] ^= Hc[row]
                pivot_cols.append(col)
                row += 1
                col += 1
                found = True
                break
        if not found:
            col += 1

    if row < m:
        raise ValueError("Could not find m independent pivot columns. H may be rank-deficient.")

    # Now pivot columns are in positions pivot_cols (they are the first m columns of Hc)
    # Build final column order: move those first m columns to the end (parity columns)
    # perm currently maps new positions -> original columns. Move first m entries to the end.
    new_col_order = np.concatenate([perm[m:], perm[:m]])  # length n
    # Apply this final permutation to original H (so H_perm = H_orig[:, new_col_order])
    H_perm = H_orig[:, new_col_order]

    # Partition H_perm into [A | B] where B is m x m (parity block at right)
    A = H_perm[:, :k].astype(np.uint8)   # m x k
    B = H_perm[:, k:].astype(np.uint8)   # m x m

    # Invert B over GF(2)
    B_inv = gf2_inverse(B)

    # P = B_inv * A  (m x k)
    P = (B_inv.dot(A) % 2).astype(np.uint8)

    # Generator in permuted column order: G_perm = [I_k | (P^T)]  size k x n
    G_left = np.eye(k, dtype=np.uint8)
    G_right = P.T  # k x m
    G_perm = np.concatenate([G_left, G_right], axis=1)  # shape k x n

    # Map G_perm columns back to original column ordering
    inv_perm = np.argsort(new_col_order)  # inv_perm[orig_index] = index_in_new
    G_full = G_perm[:, inv_perm]

    return G_full, H_perm, new_col_order

# ---------- MAIN ----------
def main():
    # load protomatrix
    proto = np.loadtxt(PROTO_FILE, dtype=int)
    M, N = proto.shape
    print(f"Loaded protomatrix {PROTO_FILE} with shape {M}x{N}; Z={Z}")

    # Expand to full H
    H = expand_protomatrix(proto, Z)
    m, n = H.shape
    k = n - m
    print(f"Expanded H: shape {m} x {n}  (m={m}, n={n}, k={k})")

    # Compute G
    try:
        G, Hperm, col_order = compute_generator_from_H(H)
    except ValueError as e:
        print("Error while computing generator matrix:", e)
        return

    print(f"Computed G: shape {G.shape} (k x n)")

    # Verify H * G^T == 0 mod 2
    check = (H.dot(G.T) % 2)
    max_entry = int(check.max())
    if max_entry == 0:
        print("Verification PASS: H * G^T == 0 (mod 2).")
    else:
        print("Verification FAIL: H * G^T has non-zero entries (max entry = {}).".format(max_entry))
        # show some failing positions
        fail_positions = np.argwhere(check != 0)
        print("Example failing positions (row, gen_row):", fail_positions[:10])

    # Save matrices to disk
    np.save(os.path.join(OUT_DIR, "H_wimax_z{0}.npy".format(Z)), H)
    np.save(os.path.join(OUT_DIR, "G_wimax_z{0}.npy".format(Z)), G)
    # also as text (CSV)
    np.savetxt(os.path.join(OUT_DIR, "H_wimax_z{0}.txt".format(Z)), H, fmt='%d', delimiter=',')
    np.savetxt(os.path.join(OUT_DIR, "G_wimax_z{0}.txt".format(Z)), G, fmt='%d', delimiter=',')

    print("Saved H and G to:", OUT_DIR)

if __name__ == "__main__":
    main()


Loaded protomatrix ./wman_N0576_R34_z24.txt with shape 6x24; Z=24
Expanded H: shape 144 x 576  (m=144, n=576, k=432)
Computed G: shape (432, 576) (k x n)
Verification PASS: H * G^T == 0 (mod 2).
Saved H and G to: ./
